#ALPHAFOLD RESULTS TO FEATURES TO INPUT TO GNN

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from pathlib import Path


In [ ]:
from zipfile import ZipFile
import os


directory = '/content/drive/MyDrive/alphafold_results'
 
# iterate over files in that directory
files = Path(directory).glob('*')
for file in files:
    # opening the zip file in READ mode
    with ZipFile(file, 'r') as zip:
      # storing all the contents of the zip file
        zipped_files = zip.namelist()
        # extracting only the pdb to a folder
        for pdbfile in zipped_files:
          if(pdbfile[-4:]==".pdb"):
            print(pdbfile)
            zip.extract(pdbfile,"/content/drive/MyDrive/Extracted_PDBs")
            break
    zip.close()
      


test_46b15_unrelaxed_rank_1_model_1.pdb
test_f7da8_unrelaxed_rank_1_model_1.pdb
test_85408_unrelaxed_rank_1_model_1.pdb
test_d65b5_unrelaxed_rank_1_model_1.pdb
test_35045_unrelaxed_rank_1_model_1.pdb
test_342b4_unrelaxed_rank_1_model_1.pdb
test_f0c24_unrelaxed_rank_1_model_1.pdb
test_36d17_unrelaxed_rank_1_model_1.pdb
test_c0e83_unrelaxed_rank_1_model_1.pdb
test_9efd6_unrelaxed_rank_1_model_1.pdb
test_9a340_unrelaxed_rank_1_model_1.pdb
test_9e089_unrelaxed_rank_1_model_1.pdb
test_f6e3a_unrelaxed_rank_1_model_1.pdb
test_fb06a_unrelaxed_rank_1_model_1.pdb
test_04206_unrelaxed_rank_1_model_1.pdb
test_e12c8_unrelaxed_rank_1_model_1.pdb
test_69d4b_unrelaxed_rank_1_model_1.pdb
test_c77b9_unrelaxed_rank_1_model_1.pdb
test_90296_unrelaxed_rank_1_model_1.pdb
test_b667c_unrelaxed_rank_1_model_1.pdb
test_2ff52_unrelaxed_rank_1_model_1.pdb


In [ ]:
import numpy as np
def preprocess_to_feature_input(filename):
  with open (filename, "r") as myfile:
    data = myfile.read().splitlines()
    
  unique_atoms = np.unique([i.split()[0] for i in data] + [i.split()[1] for i in data])
  atoms2id ={i:ind for ind, i in enumerate(unique_atoms)}
  atoms2id
  id2atoms = {ind:i for ind, i in enumerate(unique_atoms)}
  deg_temp = np.unique([i.split()[0] for i in data] + [i.split()[1] for i in data], return_counts = True)
  atom_degree = {atoms2id[deg_temp[0][i]]:deg_temp[1][i] for i in range(len(deg_temp[0]))}
  edges = [[atoms2id[i.split()[0]], atoms2id[i.split()[1]]] for i in data]
  bond_length = {str(atoms2id[i.split()[0]])+"_" + str(atoms2id[i.split()[1]]):float(i.split()[5][:-1]) for i in data}
  id2atomName = {atoms2id[i]: i.split(":")[-1] for i in atoms2id}

  res = {"edges": edges, "features_degree": atom_degree, "bond_length" : bond_length, "atomName":id2atomName, "id2atoms":id2atoms}
  np.save(f"{filename[:-4]}_out.npy", res)

  return res
  


In [ ]:
x = str(file)
x[-16:-11]

'2ff52'

In [ ]:
!pip install graphein

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 636 kB 58.2 MB/s 
     |████████████████████████████████| 58 kB 7.3 MB/s 
     |████████████████████████████████| 197 kB 68.7 MB/s 
     |████████████████████████████████| 72 kB 1.4 MB/s 
     |████████████████████████████████| 878 kB 52.4 MB/s 
     |████████████████████████████████| 237 kB 72.5 MB/s 
     |████████████████████████████████| 2.6 MB 70.4 MB/s 
     |████████████████████████████████| 11.2 MB 50.2 MB/s 
     |████████████████████████████████| 47 kB 5.3 MB/s 
     |████████████████████████████████| 48 kB 6.3 MB/s 
     |████████████████████████████████| 144 kB 65.5 MB/s 
     |████████████████████████████████| 965 kB 53.0 MB/s 
     |████████████████████████████████| 6.0 MB 50.0 MB/s 
     |████████████████████████████████| 254 kB 67.5 MB/s 
     |████████████████████████████████| 127 kB 74.0 MB/s 
     |█████

In [ ]:
from graphein.protein.graphs import construct_graph
from graphein.protein.edges.atomic import add_atomic_edges
from graphein.protein.config import ProteinGraphConfig
import networkx as nx

params_to_change = {"granularity": "atom", "edge_construction_functions": [add_atomic_edges]}

config = ProteinGraphConfig(**params_to_change)
config.dict()


directory = '/content/drive/MyDrive/Extracted_PDBs'
 
# iterate over files in that directory
files = Path(directory).glob('*')
for file in files:
  # print(file)
  if(str(file)[-4:]==".pdb"):
    print(str(file)[-34:-29])
    g = construct_graph(config=config, pdb_path=str(file))
    txt_filename = 'pdb_text'+str(file)[-34:-29]+'.txt'
    print(txt_filename)
    nx.write_edgelist(g, txt_filename)
    gh = preprocess_to_feature_input(txt_filename)
    print(str(file)+" Done")

ModuleNotFoundError: ignored